# 🧩 `parse_markdown_examples_to_json(text: str)`

Парсит текст с примерами, содержащими блоки `**Input:**` и `**Output:**`, и возвращает их в виде словаря. Этот формат используется у LeetCode.

---

## 📥 Аргументы

| Параметр | Тип   | Описание                             |
|----------|-------|--------------------------------------|
| `text`   | `str` | Текст, содержащий примеры в формате: |
| -        |       | `**Example N:**`                     |
| -        |       | `**Input:** <входные данные>`        |
| -        |       | `**Output:** <выходные данные>`      |

---

## 📤 Возвращает

Тип: `dict`

Структура возвращаемого словаря:

```json
{
  "examples": [
    {
      "input": {...},     // Входные данные (dict)
      "output": ...       // Выходные данные (любой тип)
    }
  ]
}
```

---


# 🔻 Вставьте пример для конвертирования в JSON

In [1]:
example_block = """
**Example 1:**

![](https://assets.leetcode.com/uploads/2021/02/19/rev1ex1.jpg)

**Input:** head = [1,2,3,4,5]
**Output:** [5,4,3,2,1]

**Example 2:**

![](https://assets.leetcode.com/uploads/2021/02/19/rev1ex2.jpg)

**Input:** head = [1,2]
**Output:** [2,1]

**Example 3:**

**Input:** head = []
**Output:** []
"""

## 1. Функция-обработчик

In [2]:
import re
import ast
import json

def parse_markdown_examples_to_json(text: str):
    examples = []
    raw_blocks = re.split(r"\*\*Example \d+:\*\*", text)

    for idx, block in enumerate(raw_blocks, 1):
        prefix = f"Example {idx}"

        # Попытка найти input
        input_lines = re.findall(r"\*\*Input:\*\*\s*(.*?)(?=\n\*\*|$)", block, re.DOTALL)
        if not input_lines:
            continue
        input_lines = [line.strip().rstrip(",") for line in input_lines[0].strip().splitlines() if line.strip()]
        try:
            if len(input_lines) == 1:
                # Случай s=["MyQueue"], x=[[]]
                input_expr = "{" + re.sub(r"(\w+)\s*=", r'"\1":', input_lines[0]) + "}"
                input_data = ast.literal_eval(input_expr)
            else:
                # Случай ["MyQueue"], [[], []]
                input_data = {"args": [ast.literal_eval(line) for line in input_lines]}
        except Exception as e:
            print(f"{prefix} ⚠ Ошибка в input: {e}")
            continue

        # Попытка найти output
        output_match = re.search(r"\*\*Output:\*\*\s*(.*?)\s*(?:\n|$)", block)
        if not output_match:
            continue

        output_text = output_match.group(1).strip()
        output_text = output_text.replace("null", "None").replace("true", "True").replace("false", "False")

        try:
            output_data = ast.literal_eval(output_text)
        except Exception as e:
            print(f"{prefix} ⚠ Ошибка в output: {e}")
            continue

        examples.append({
            "input": input_data,
            "output": output_data
        })

    return {"examples": examples}


## 2. Парсим примеры из текста

In [3]:
parsed = parse_markdown_examples_to_json(example_block)
print(parsed)

{'examples': [{'input': {'head': [1, 2, 3, 4, 5]}, 'output': [5, 4, 3, 2, 1]}, {'input': {'head': [1, 2]}, 'output': [2, 1]}, {'input': {'head': []}, 'output': []}]}


## 3. Преобразуем результат в JSON-строку с красивым форматированием

In [4]:
pretty = json.dumps(parsed, indent=2, ensure_ascii=False)
print(pretty)

{
  "examples": [
    {
      "input": {
        "head": [
          1,
          2,
          3,
          4,
          5
        ]
      },
      "output": [
        5,
        4,
        3,
        2,
        1
      ]
    },
    {
      "input": {
        "head": [
          1,
          2
        ]
      },
      "output": [
        2,
        1
      ]
    },
    {
      "input": {
        "head": []
      },
      "output": []
    }
  ]
}


## 4. Убираем лишние пробелы и переносы строк внутри массивов

In [5]:
pretty = re.sub(r'\[\s+([^\[\]]+?)\s+\]', lambda m: '[' + m.group(1).replace('\n', '').replace(' ', '') + ']', pretty)
print(pretty)

{
  "examples": [
    {
      "input": {
        "head": [1,2,3,4,5]
      },
      "output": [5,4,3,2,1]
    },
    {
      "input": {
        "head": [1,2]
      },
      "output": [2,1]
    },
    {
      "input": {
        "head": []
      },
      "output": []
    }
  ]
}


# 🔺 Заберите результат в буфер обмена